# Aula 5: Chamada de Ferramentas (Tool Calling) com Pydantic e OpenAI

Nesta aula, você aprenderá a usar modelos Pydantic para definir ferramentas para a API de "tool calling" da OpenAI. Você verá como reutilizar seus modelos existentes para criar definições de ferramentas robustas e validadas, e como lidar com as chamadas de ferramentas em seu código Python. Esta lição se baseia nos seus modelos `UserInput` e `CustomerQuery` das aulas anteriores.

### Conexão com LangChain e LangGraph
Esta aula é a base para a construção de Agentes de IA. Enquanto o estado em um grafo do LangGraph é frequentemente gerenciado com `TypedDict`, as **ferramentas (tools)** que o agente utiliza precisam de um esquema claro para que o LLM saiba como usá-las. O Pydantic é a ferramenta perfeita para definir esses esquemas, garantindo que os dados passados para suas ferramentas sejam sempre válidos e bem estruturados. Dominar esta técnica é fundamental para criar agentes confiáveis.

Ao final desta lição, você será capaz de:
- Usar modelos Pydantic para definir esquemas de ferramentas para a API da OpenAI.
- Registrar suas ferramentas na API usando um esquema validado.
- Lidar com chamadas de ferramentas e validar os argumentos com Pydantic.
- Integrar fluxos de trabalho orientados por LLM com suas próprias funções e fontes de dados em Python.

### Célula 2: Importações e Configuração

**Explicação Didática:**
Toda aplicação começa com a importação das bibliotecas necessárias. O `instructor` "remenda" o cliente da OpenAI, adicionando a capacidade de solicitar respostas em um formato Pydantic específico através do parâmetro `response_model`. Usamos `dotenv` para carregar nossa chave de API de um arquivo `.env` de forma segura.

**Ação:** Crie um arquivo chamado `.env` no mesmo diretório deste notebook e adicione a seguinte linha:
```
OPENAI_API_KEY="sua_chave_de_api_aqui"
```

In [ ]:
# Célula de importação e configuração.
# Se você encontrar um ModuleNotFoundError, significa que a biblioteca não está instalada.
# Para instalar o que precisamos, execute no seu terminal:
# pip install openai pydantic instructor python-dotenv

import json
from datetime import date, datetime
from typing import List, Literal, Optional
import os

# A biblioteca 'instructor' é a chave para conectar Pydantic e OpenAI
import instructor
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, EmailStr, Field, field_validator

# Carrega as variáveis de ambiente (ex: OPENAI_API_KEY) do arquivo .env
load_dotenv()

# Cria o cliente da OpenAI. O instructor vai "remendá-lo".
# Isso adiciona o parâmetro 'response_model' às chamadas do cliente.
client = instructor.patch(OpenAI(api_key=os.getenv("OPENAI_API_KEY")))

### Célula 3: Modelos Pydantic de Entrada

**Explicação Didática:**
Aqui definimos nossas estruturas de dados com Pydantic. `UserInput` representa os dados brutos que recebemos do usuário. `CustomerQuery` herda de `UserInput` e adiciona campos que serão preenchidos pelo LLM para classificar a consulta. O uso de `Field` nos permite adicionar descrições, que são cruciais para que o LLM entenda o propósito de cada campo. O `@field_validator` é um ótimo exemplo do poder do Pydantic, garantindo que um `order_id`, se fornecido, siga um formato específico.

In [ ]:
# Define o modelo para a entrada do usuário
class UserInput(BaseModel):
    name: str = Field(..., description="Nome do usuário")
    email: EmailStr = Field(..., description="Endereço de e-mail do usuário")
    query: str = Field(..., description="A pergunta/consulta do usuário")
    order_id: Optional[str] = Field(
        None, description="ID do pedido, se disponível (formato: ABC-12345)"
    )
    purchase_date: Optional[date] = Field(None, description="Data da compra")

    # Valida o formato do order_id (ex: ABC-12345)
    @field_validator("order_id")
    def validate_order_id(cls, order_id):
        import re

        if order_id is None:
            return order_id
        pattern = r"^[A-Z]{3}-\d{5}$"
        if not re.match(pattern, order_id):
            raise ValueError(
                "O order_id deve estar no formato ABC-12345 "
                "(3 letras maiúsculas, traço, 5 dígitos)"
            )
        return order_id

# Define o modelo para a consulta classificada pelo LLM
class CustomerQuery(UserInput):
    priority: str = Field(..., description="Nível de prioridade: baixo, médio, alto")
    category: Literal["refund_request", "information_request", "other"] = Field(
        ..., description="Categoria da consulta"
    )
    is_complaint: bool = Field(..., description="Indica se a consulta é uma reclamação")
    tags: List[str] = Field(..., description="Tags de palavras-chave relevantes")

### Célula 4: Função de Classificação com LLM

**Explicação Didática:**
Esta função é nosso primeiro passo de "inteligência". Ela recebe a entrada do usuário (já como um objeto Pydantic validado) e usa o LLM da OpenAI para enriquecê-la, transformando um `UserInput` em um `CustomerQuery`. Graças ao `instructor`, podemos passar `response_model=CustomerQuery` diretamente na chamada da API. Isso instrui o LLM a formatar sua resposta de acordo com o esquema do nosso modelo, e o `instructor` garante que o resultado seja uma instância validada do `CustomerQuery`.

In [ ]:
# Função para chamar o LLM e criar uma instância de CustomerQuery
def create_customer_query_openai(user_input: UserInput) -> CustomerQuery:
    """
    Usa o GPT-4o para analisar a entrada do usuário e extrair/inferir os
    campos adicionais para criar um CustomerQuery estruturado.
    """
    # O 'response_model' é a mágica do instructor. Ele formata o prompt
    # para o LLM e valida a saída, retornando um objeto Pydantic.
    customer_query = client.chat.completions.create(
        model="gpt-4o",
        response_model=CustomerQuery,
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em classificação de consultas de clientes. Analise a consulta do usuário e preencha os campos de prioridade, categoria, reclamação e tags.",
            },
            {
                "role": "user",
                "content": user_input.model_dump_json(indent=2),
            },
        ],
    )
    print("CustomerQuery gerado com sucesso!")
    return customer_query

### Célula 5: Testando a Classificação

**Explicação Didática:**
Agora vamos testar a função que acabamos de criar. Fornecemos um JSON de exemplo, primeiro validamos com o nosso modelo `UserInput` e depois passamos para a função `create_customer_query_openai`. O resultado impresso mostrará o objeto Pydantic completo, com os campos `priority`, `category`, etc., preenchidos pelo LLM.

In [ ]:
# JSON com a entrada de exemplo do usuário
user_input_json = """
{
    "name": "Joe User",
    "email": "joe@example.com",
    "query": "Quando posso esperar a entrega dos fones de ouvido que pedi?",
    "order_id": "ABC-12345",
    "purchase_date": "2025-12-01"
}
"""

# 1. Valida a entrada bruta do usuário
user_input = UserInput.model_validate_json(user_input_json)

# 2. Usa o LLM para classificar e enriquecer os dados
customer_query = create_customer_query_openai(user_input)

# 3. Imprime o resultado
print("\n--- Objeto CustomerQuery Gerado ---")
print(customer_query.model_dump_json(indent=2))

### Célula 6: Modelos de Input para Ferramentas

**Explicação Didática:**
Aqui entramos no núcleo do "Tool Calling". Assim como definimos modelos para a entrada do usuário, definimos modelos Pydantic para os **argumentos** de cada ferramenta que nosso agente poderá usar. `FAQLookupArgs` define que a ferramenta de busca no FAQ precisa de uma `query` e uma lista de `tags`. `CheckOrderStatusArgs` exige um `order_id` e um `email`. Essas definições serão convertidas em JSON Schema para que o LLM saiba exatamente o que enviar ao chamar a ferramenta.

In [ ]:
# Modelo de argumentos para a ferramenta de busca no FAQ
class FAQLookupArgs(BaseModel):
    query: str = Field(..., description="A pergunta original do usuário")
    tags: List[str] = Field(
        ..., description="Tags de palavras-chave relevantes extraídas da consulta do cliente"
    )

# Modelo de argumentos para a ferramenta de verificação de status do pedido
class CheckOrderStatusArgs(BaseModel):
    order_id: str = Field(..., description="O ID do pedido do cliente (formato: ABC-12345)")
    email: EmailStr = Field(..., description="O endereço de e-mail do cliente")

    @field_validator("order_id")
    def validate_order_id(cls, order_id):
        import re
        pattern = r"^[A-Z]{3}-\d{5}$"
        if not re.match(pattern, order_id):
            raise ValueError("O formato do order_id deve ser ABC-12345")
        return order_id

### Célula 7: Implementação das Ferramentas e "Bancos de Dados"

**Explicação Didática:**
Estas são as funções Python que nossas ferramentas irão de fato executar. `lookup_faq_answer` e `check_order_status` simulam a interação com sistemas externos (um banco de dados de FAQs e um de pedidos, que criamos como exemplos). O LLM não vê este código; ele apenas sabe o nome das funções e os argumentos que elas esperam (definidos pelos nossos modelos Pydantic na célula anterior).

In [ ]:
# Base de dados de exemplo para o FAQ
faq_db = [
    {
        "question": "Como posso resetar minha senha?",
        "answer": "Para resetar, clique em 'Esqueci a Senha' na página de login e siga as instruções.",
        "keywords": ["senha", "resetar", "conta"],
    },
    {
        "question": "Quanto tempo leva a entrega?",
        "answer": "A entrega padrão leva de 3 a 5 dias úteis. Você pode rastrear seu pedido no seu painel.",
        "keywords": ["entrega", "envio", "pedido", "rastreamento", "headphones"],
    },
]

# Base de dados de exemplo para os pedidos
order_db = {
    "ABC-12345": {
        "status": "enviado",
        "estimated_delivery": "2025-12-05",
        "email": "joe@example.com",
    },
}

# Implementação da função da ferramenta de busca no FAQ
def lookup_faq_answer(args: FAQLookupArgs) -> str:
    # ... (código da função original, é uma boa implementação)
    query_words = set(word.lower() for word in args.query.split())
    tag_set = set(tag.lower() for tag in args.tags)
    best_match = None
    best_score = 0
    for faq in faq_db:
        keywords = set(k.lower() for k in faq["keywords"])
        score = len(keywords & tag_set) + len(keywords & query_words)
        if score > best_score:
            best_score = score
            best_match = faq
    if best_match and best_score > 0:
        return best_match["answer"]
    return "Desculpe, não encontrei uma resposta no FAQ para sua pergunta."


# Implementação da função da ferramenta de status de pedido
def check_order_status(args: CheckOrderStatusArgs) -> dict:
    # ... (código da função original)
    order = order_db.get(args.order_id)
    if not order:
        return {"status": "não encontrado"}
    if args.email.lower() != order.get("email", "").lower():
        return {"status": "e-mail não corresponde"}
    return {"status": order["status"], "entrega_estimada": order["estimated_delivery"]}

### Célula 8: Definição dos Esquemas das Ferramentas para a API

**Explicação Didática:**
Esta lista `tool_definitions` é o que passaremos para a API da OpenAI. Cada item descreve uma ferramenta. O Pydantic nos ajuda imensamente aqui: `FAQLookupArgs.model_json_schema()` gera automaticamente o dicionário JSON Schema que a API precisa para o campo `parameters`. Isso é muito mais limpo e seguro do que escrevê-lo manualmente.

In [ ]:
# Define as ferramentas que a API da OpenAI poderá chamar
tool_definitions = [
    {
        "type": "function",
        "function": {
            "name": "lookup_faq_answer",
            "description": "Busca uma resposta no FAQ combinando tags e palavras-chave da consulta.",
            "parameters": FAQLookupArgs.model_json_schema(),
        },
    },
    {
        "type": "function",
        "function": {
            "name": "check_order_status",
            "description": "Verifica o status do pedido de um cliente.",
            "parameters": CheckOrderStatusArgs.model_json_schema(),
        },
    },
]

### Célula 9: Modelo Pydantic de Saída Final

**Explicação Didática:**
Finalmente, definimos o modelo da nossa saída final e estruturada: o `SupportTicket`. Ele herda do `CustomerQuery` (reutilizando os campos já preenchidos) e adiciona informações de ação, como a recomendação do LLM e os resultados das ferramentas que foram chamadas.

In [ ]:
# Define o modelo Pydantic para a saída final
class OrderDetails(BaseModel):
    status: str
    entrega_estimada: Optional[str] = None

class SupportTicket(CustomerQuery):
    recommended_next_action: Literal[
        "escalate_to_agent", "send_faq_response", "send_order_status", "no_action_needed"
    ] = Field(..., description="A próxima ação recomendada pelo LLM para o suporte")
    order_details: Optional[OrderDetails] = Field(
        None, description="Detalhes do pedido se a ação for 'send_order_status'"
    )
    faq_response: Optional[str] = Field(
        None, description="Resposta do FAQ se a ação for 'send_faq_response'"
    )
    creation_date: datetime = Field(
        default_factory=datetime.now, description="Data e hora de criação do ticket"
    )

### Célula 10: Fluxo de Orquestração com Múltiplas Chamadas

**Explicação Didática:**
Esta é a função principal que orquestra todo o processo.
1.  **Primeira Chamada (Decisão):** Envia a consulta do usuário e a lista de ferramentas disponíveis para o OpenAI. O LLM decide se pode responder diretamente ou se precisa chamar uma ferramenta.
2.  **Execução da Ferramenta:** Se o LLM solicitar uma chamada de ferramenta, nosso código Python executa a função correspondente (`lookup_faq_answer` ou `check_order_status`).
3.  **Segunda Chamada (Síntese):** Enviamos uma nova mensagem para o LLM, desta vez incluindo o resultado da ferramenta. O LLM então usa essa nova informação para gerar a resposta final e preencher nosso `SupportTicket`.
4.  **Geração Estruturada:** Usamos o `instructor` novamente na chamada final para garantir que a saída seja um objeto `SupportTicket` validado.

In [ ]:
def run_support_workflow(customer_query: CustomerQuery) -> SupportTicket:
    """
    Orquestra o fluxo completo: decide a ação, chama ferramentas se necessário,
    e gera o ticket de suporte final.
    """
    print("\n--- Passo 1: Decidindo a próxima ação ---")
    # O prompt do sistema instrui o LLM sobre seu papel e as ferramentas disponíveis
    messages = [
        {
            "role": "system",
            "content": "Você é um agente de suporte. Use as ferramentas disponíveis para obter informações adicionais (como status de pedido ou respostas de FAQ) antes de decidir a próxima ação.",
        },
        {"role": "user", "content": f"Consulta do cliente: {customer_query.model_dump_json()}"},
    ]

    # Primeira chamada: O LLM decide se precisa de uma ferramenta
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tool_definitions,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # Adiciona a resposta da IA (que pode incluir pedidos de ferramenta) ao histórico
    messages.append(response_message)

    if tool_calls:
        print(f"\n--- Passo 2: LLM solicitou {len(tool_calls)} chamada(s) de ferramenta ---")
        available_functions = {
            "lookup_faq_answer": lookup_faq_answer,
            "check_order_status": check_order_status,
        }
        # Executa cada ferramenta solicitada
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            # Valida os argumentos com Pydantic antes de chamar a função
            if function_name == "lookup_faq_answer":
                function_args_model = FAQLookupArgs.model_validate_json(tool_call.function.arguments)
            else:
                function_args_model = CheckOrderStatusArgs.model_validate_json(tool_call.function.arguments)

            print(f"Executando: {function_name}({function_args_model.model_dump()})")
            function_response = function_to_call(function_args_model)

            # Adiciona o resultado da ferramenta ao histórico de mensagens
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(function_response),
                }
            )
        
        print("\n--- Passo 3: Gerando o ticket final com o resultado da ferramenta ---")
        # Segunda chamada: O LLM usa o resultado da ferramenta para gerar a saída final
        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            response_model=SupportTicket, # Usando instructor para a saída final
        )
        return final_response
    else:
        print("\n--- Passo 2 e 3: Nenhuma ferramenta foi chamada. Gerando ticket diretamente. ---")
        # Se nenhuma ferramenta foi necessária, pedimos o ticket final mesmo assim
        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            response_model=SupportTicket,
        )
        return final_response

### Célula 11: Executando o Fluxo Completo

**Explicação Didática:**
Aqui, executamos o fluxo completo com a consulta do nosso cliente. O resultado final será um `SupportTicket` completo e bem estruturado, que poderia ser facilmente salvo em um banco de dados ou enviado para outro sistema.

In [ ]:
# Executa o fluxo completo
support_ticket = run_support_workflow(customer_query)

# Imprime o ticket de suporte final e validado
print("\n--- Ticket de Suporte Final Gerado ---")
print(support_ticket.model_dump_json(indent=2))

### Célula 12: Conclusão

**Explicação Didática:**
Para finalizar, um resumo do que aprendemos. Esta célula reforça a importância do Pydantic na criação de sistemas de IA confiáveis, conectando a validação de dados de entrada, a definição de ferramentas e a geração de saídas estruturadas, que são os blocos de construção para os agentes que você está estudando em LangChain e LangGraph.

--- 

## Conclusão

Nesta lição, você aprendeu a usar modelos Pydantic para definir ferramentas para a API de "tool calling" da OpenAI e a construir um fluxo de trabalho de suporte ao cliente do início ao fim. Você viu como reutilizar modelos para criar definições de ferramentas robustas, lidar com as chamadas de API, executar a lógica correspondente em Python e, finalmente, gerar uma saída estruturada e validada.

Esta abordagem demonstra o poder do Pydantic na criação de pipelines de IA confiáveis. A validação em cada etapa – da entrada do usuário, passando pelos argumentos das ferramentas, até a saída final – é o que permite que Agentes de IA complexos, como os construídos com LangChain e LangGraph, operem de forma segura e previsível.